In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
import seaborn as sns

In [2]:
all_data = pd.read_csv('all_data_with_intervals.csv')[['Bundesland','week_no','summer_vac','active_lockdown','total_population']]
populations = all_data.groupby('Bundesland')['total_population'].sum().reset_index()
intervals = all_data.drop(columns='total_population').drop_duplicates()

In [3]:
rki_weekly = pd.read_csv('rki_weekly.csv')
rki_weekly['Bundesland'] = rki_weekly.districtId // 1000
state_wise = rki_weekly.groupby(['Bundesland','week_no'])['AnzahlFall'].sum().reset_index().merge(populations)
state_wise['cases_per_cap'] = state_wise.AnzahlFall / state_wise.total_population

In [22]:
plot_data = pd.merge(state_wise, intervals, how='outer')
plot_data.fillna(value=False,inplace=True)
plot_data = plot_data[plot_data.week_no.between(6,39)].copy()#our model was only until week 38, with RF looking ahead 1 week
plot_lockdown = plot_data[plot_data.active_lockdown].copy()
plot_summer = plot_data[plot_data.summer_vac].copy()

In [59]:
# Y_VAR = 'cases_per_cap'
Y_VAR = 'AnzahlFall'

fig = go.Figure()

value_repo = {}

stackr = {}
for i in np.unique(plot_data.week_no):
    stackr[i] = 0

bl_range = range(1,17) #parametrized here in order to change order etc
    
for bl in bl_range:
    
    df_bl = plot_data[plot_data.Bundesland==bl]

    X_all = []
    Y_all = []
    X_lockdown = []
    Y_lockdown = []
    X_summer = []
    Y_summer = []

    for i, val in df_bl.iterrows():

        x_ = val.week_no
        stackr[x_] += val[Y_VAR]
        y_ = stackr[x_]

        X_all.append(x_)
        Y_all.append(y_)

        if val.active_lockdown:
            X_lockdown.append(x_)
            Y_lockdown.append(y_)

        if val.summer_vac:
            X_summer.append(x_)
            Y_summer.append(y_)
            
        
    value_repo[bl] = {'all':pd.Series(Y_all,index=X_all),
                      'Active Lockdown':pd.Series(Y_lockdown,index=X_lockdown),
                      'Summer Vacation':pd.Series(Y_summer,index=X_summer)}

    
def add_areas(bl, timeslice, color):
    
    suppl_line_attrs = {'mode':'lines','line_width':0,'hoverinfo':'none'}
    
    not_first = bl != bl_range[0]
    
    if not_first:#this creates invisible lines for the areas to fill down towards, otherwise area borders are not vertical
        last_y = value_repo[bl-1]['all']
        last_y = last_y[last_y.index.isin(vals[timeslice].index)]
        
        fig.add_trace(go.Scatter(x=vals[timeslice].index,
                                 y=last_y,
                                 fill=None,
                                 showlegend=False,
                                 **suppl_line_attrs))
    
    fig.add_trace(go.Scatter(x=vals[timeslice].index,
                             y=vals[timeslice],
                             fill='tonexty' if not_first else 'tozeroy',
                             #explicit tozeroy, because we add traces per BL multiple times,
                             name=None if not_first else timeslice,
                             showlegend= not not_first,
                             line_color=color,
                             **suppl_line_attrs))
    
for bl in bl_range: #not iter directly over dict to potentially change order
    vals = value_repo[bl]
    #now summer
    add_areas(bl, 'Summer Vacation', 'blue')
    
    #now the lockdown
    add_areas(bl, 'Active Lockdown', 'red')
    
    
    #now rest
    fig.add_trace(go.Scatter(x=vals['all'].index,
                             y=vals['all'],
                             fill=None,
                             showlegend=False,
                             line_color='black',
                             line_width=.9,
                             mode='lines',
                             hoverinfo='none'))

fig.update_layout(
    title=dict(
        text="Weekly New Covid-19 Cases in Germany by Province",
        xanchor='center',
        x=.5),
    xaxis_title="2020 Calendar Week",
    yaxis_title="Weekly New Cases",
    legend_title="Period of Analysis:   ",
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="center",
        x=0.5
    ),
    font=dict(
        size=18,
        color="black"
    )
)
fig.show()
